In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [2]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [3]:
df = pd.read_csv('../datasets/helpdesk_customer_tickets.csv')

In [4]:
df = df.drop(['id','answer','priority','tag_5','tag_6','tag_7','tag_8','tag_9'],axis=1)
df.head()

,subject,body,type,queue,language,business_type,tag_1,tag_2,tag_3,tag_4
0,Anfrage zu den Spezifikationen und Anpassungso...,Sehr geehrtes Support-Team des Tech Online Sto...,Request,Customer Service,de,Tech Online Store,Product Support,Sales Inquiry,Technical Guidance,General Inquiry
1,Déconnexions fréquentes et plantages,Le client signale des déconnexions fréquentes ...,Incident,Product Support,fr,Software Development Company,Technical Support,Software Bug,Service Disruption,System Crash
2,Problema de sonido Dell XPS,"Problema con el sonido, manejando como devoluc...",Problem,Returns and Exchanges,es,Tech Online Store,Returns and Exchanges,Product Support,Customer Service,Refund Request
3,Assistance requise pour la configuration du ta...,"Cher support client,\r\n\r\nNotre client, <nam...",Request,Product Support,fr,Software Development Company,Technical Support,Product Support,General Inquiry,Problem Resolution
4,Urgente: Assistência Imediata Necessária para ...,Caro Suporte ao Cliente da Firma de Consultori...,Incident,Human Resources,pt,IT Consulting Firm,Urgent Issue,Payroll Issue,Technical Support,Service Disruption


In [10]:
bdf = BaseDatasetTransform(df,target='queue',text_columns=['subject','body'],)

In [11]:
X_train, X_test, y_train, y_test ,categorical_features = bdf.fit_transform()

Series([], dtype: int64)

В колонке(ах) ['queue'] нет пропущенных значений


c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|██████████| 19/19 [01:12<00:00,  3.82s/it]

В наборе данных нет пропущенных значений
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['type', 'language', 'business_type', 'tag_1', 'tag_2', 'tag_3',
       'tag_4'],
      dtype='object')

Числовые колонки:
Index(['1_feature', '2_feature', '3_feature', '4_feature', '5_feature',
       '6_feature', '7_feature', '8_feature', '9_feature', '10_feature',
       ...
       '759_feature', '760_feature', '761_feature', '762_feature',
       '763_feature', '764_feature', '765_feature', '766_feature',
       '767_feature', '768_feature'],
      dtype='object', length=768)


In [7]:
lgbm_model = lgb.LGBMClassifier(verbose=-1)

columns_to_delete = []  

unique_classes = len(set(y_train))
print(unique_classes)

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='multiclass',  
    num_class=unique_classes, 
    metric='multi_error' 
)

1


In [8]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Dmitriy.Burnashev\AppData\Local\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['type',
                                                                  'language',
                                                                  'business_type',
                                                                  'tag_1',
                                                                  'tag_2',
                                                                  'tag_3',
                                                                  'tag_4'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(metric='multi_error',
                                                                 num_class=1,
                                                                 objective='multiclass',
                                                                 verbose=-1)))

In [9]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.544444
1,precision,0.788240
2,recall,0.503789
3,f1,0.572775
